Trabajo con Datos con el Core: Inserción de Filas
====

* Última modificación: 12 de septiembre de 2022.

Preparación
----

In [1]:
from sqlalchemy import MetaData

metadata_obj = MetaData()

In [2]:
from sqlalchemy import Column, Integer, String, Table

user_table = Table(
    "user_account",
    metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("name", String(30)),
    Column("fullname", String),
)

In [3]:
from sqlalchemy import ForeignKey

address_table = Table(
    "address",
    metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("user_id", ForeignKey("user_account.id"), nullable=False),  # Clave foranea
    Column("email_address", String, nullable=False),
)

In [4]:
from sqlalchemy import create_engine

engine = create_engine(
    "sqlite+pysqlite:///:memory:",
    echo=True,
    future=True,
)

metadata_obj.create_all(engine)

2022-09-13 21:53:35,890 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-09-13 21:53:35,891 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2022-09-13 21:53:35,892 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-09-13 21:53:35,893 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("user_account")
2022-09-13 21:53:35,893 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-09-13 21:53:35,894 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2022-09-13 21:53:35,895 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-09-13 21:53:35,896 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("address")
2022-09-13 21:53:35,896 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-09-13 21:53:35,898 INFO sqlalchemy.engine.Engine 
CREATE TABLE user_account (
	id INTEGER NOT NULL, 
	name VARCHAR(30), 
	fullname VARCHAR, 
	PRIMARY KEY (id)
)


2022-09-13 21:53:35,898 INFO sqlalchemy.engine.Engine [no key 0.00063s] ()
2022-09-13 21:53:35,899 INFO sqlalchemy.engine.Engine 
C

Expresión INSERT INTO ...
---

In [5]:
from sqlalchemy import insert

stmt = insert(user_table).values(name="spongebob", fullname="Spongebob Squarepants")

#
# Objeto en memoria
#
stmt

In [6]:
#
# Impresión como una cadena de caracteres. Note que insert()
# crea automaticamente la expresión SQL para insertar datos
#
print(stmt)

INSERT INTO user_account (name, fullname) VALUES (:name, :fullname)


In [7]:
#
# Valores de los parámetros en el stmt
#
compiled = stmt.compile()
compiled.params

{'name': 'spongebob', 'fullname': 'Spongebob Squarepants'}

Ejecución de la expresión INSERT INTO ...
----

In [8]:
with engine.connect() as conn:
    result = conn.execute(stmt)
    conn.commit()

2022-09-13 21:53:36,215 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-09-13 21:53:36,216 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2022-09-13 21:53:36,216 INFO sqlalchemy.engine.Engine [generated in 0.00196s] ('spongebob', 'Spongebob Squarepants')
2022-09-13 21:53:36,218 INFO sqlalchemy.engine.Engine COMMIT


In [9]:
result.inserted_primary_key

(1,)

INSERT INTO ... con varias filas
----

In [10]:
with engine.connect() as conn:
    result = conn.execute(
        insert(user_table),
        [
            {"name": "sandy", "fullname": "Sandy Cheeks"},
            {"name": "patrick", "fullname": "Patrick Star"},
        ],
    )
    conn.commit()

2022-09-13 21:53:36,348 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-09-13 21:53:36,350 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2022-09-13 21:53:36,350 INFO sqlalchemy.engine.Engine [generated in 0.00216s] (('sandy', 'Sandy Cheeks'), ('patrick', 'Patrick Star'))
2022-09-13 21:53:36,351 INFO sqlalchemy.engine.Engine COMMIT


Expresión INSERT ... FROM SELECT
----

In [11]:
from sqlalchemy import select

select_stmt = select(user_table.c.id, user_table.c.name + "@aol.com")

insert_stmt = insert(address_table).from_select(
    ["user_id", "email_address"], select_stmt
)

print(insert_stmt)

INSERT INTO address (user_id, email_address) SELECT user_account.id, user_account.name || :name_1 AS anon_1 
FROM user_account


Expresión INSERT ... RETURNING
---

In [12]:
#
# Retorna la ultima clave primaria insertada y los valores por defecto del servidor
#
insert_stmt = insert(address_table).returning(
    address_table.c.id,
    address_table.c.email_address,
)

print(insert_stmt)

INSERT INTO address (id, user_id, email_address) VALUES (:id, :user_id, :email_address) RETURNING address.id, address.email_address


In [13]:
#
# Se puede usar para crear un INSERT .... FROM SELECT
#
select_stmt = select(user_table.c.id, user_table.c.name + "@aol.com")
insert_stmt = insert(address_table).from_select(
    ["user_id", "email_address"], select_stmt
)

print(insert_stmt.returning(address_table.c.id, address_table.c.email_address))

INSERT INTO address (user_id, email_address) SELECT user_account.id, user_account.name || :name_1 AS anon_1 
FROM user_account RETURNING address.id, address.email_address
